In [1]:
#brew install gcc@8
#git clone --recursive https://github.com/dmlc/xgboost
#cd xgboost
#vi config.mk
#uncomment and change to 
#export CC = gcc-8
#export CXX = g++-8
#change Makefile
#export CC = $(if $(shell which clang), clang, gcc-8)
#export CXX = $(if $(shell which clang++), clang++, g++-8)
#make clean_all && make -j4
#cd python-package; python setup.py install

In [2]:
#import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
#from xgboost import plot_importance
from matplotlib import pyplot as plt
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.model_selection import GridSearchCV
import warnings 
warnings.filterwarnings("ignore")
from xgboost.sklearn import XGBRegressor
import pickle

In [3]:
df = pd.read_csv('../data/processed/kaggle_train_sample.csv')
df_type = pd.read_csv('../data/processed/kaggle_column_type.csv')

FileNotFoundError: [Errno 2] File b'../data/processed/kaggle_train_sample.csv' does not exist: b'../data/processed/kaggle_train_sample.csv'

In [4]:
#Function to get target variable
def get_target(df,df_type):
    for c in df:
        if (column_type(c,df_type) == 'Flag_Continuous' or column_type(c,df_type) == 'Flag_Categorical'):
            return(c)

In [5]:
# funtion to get column type
def column_type(column_name,df_type):
    return (df_type.loc[df_type['Variable'] == column_name, 'Type'].iloc[0])

In [143]:
target_name = get_target(df,df_type)
X = df.drop(columns = target_name)
y = df[target_name]

In [110]:
X.head()

,Calendar_Year,Model_Year,Blind_Make,Cat1,Cat2,Cat3,Cat6,Cat8,Cat9,Cat10,Cat11,Cat12,OrdCat,NVCat
0,1,2,3,1,2,2,2,0,0,1,2,3,4,0
1,1,2,0,1,2,5,1,2,1,2,4,3,5,0
2,1,2,1,1,2,5,2,2,1,2,1,3,2,2
3,0,2,1,1,2,5,2,2,1,1,4,5,2,2
4,1,1,0,0,2,4,4,2,1,2,4,5,5,2


In [111]:
y.head()

0   -0.105463
1   -0.105463
2   -0.105463
3   -0.105463
4   -0.105463
Name: Claim_Amount, dtype: float64

In [124]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=1)

In [125]:
y_test

2764   -0.105463
4767   -0.105463
3814   -0.105463
3499   -0.105463
2735   -0.105463
3922   -0.105463
2701   -0.105463
1179   -0.105463
932    -0.105463
792    -0.105463
1852   -0.105463
1185   -0.105463
1724   -0.105463
4080   -0.105463
3823   -0.105463
4054   -0.105463
2721   -0.105463
3903   -0.105463
1865   -0.105463
759    -0.105463
2416   -0.105463
3670   -0.105463
4838   -0.105463
3090   -0.105463
1546   -0.105463
1132   -0.105463
4703   -0.105463
4494   -0.105463
2775   -0.105463
3185   -0.105463
          ...   
55     -0.105463
115    -0.105463
3396   -0.105463
1291   -0.105463
3451   -0.105463
2948   -0.105463
4353   -0.105463
4762   -0.105463
1720   -0.105463
2068   -0.105463
703    -0.105463
2027   -0.105463
301    -0.105463
4734   -0.105463
3936   -0.105463
560    -0.105463
1386   -0.105463
3547   -0.105463
1896   -0.105463
1535   -0.105463
4823   -0.105463
994    -0.105463
2083   -0.105463
3312   -0.105463
86     -0.105463
448    -0.105463
921    -0.105463
4087   -0.1054

In [1]:
model = XGBRegressor(max_depth=1, learning_rate=0.1, n_estimators=160, silent=True, objective='reg:linear',eval_metric='rmse')
model.fit(X_train, y_train)

# prediction
pred_test = model.predict(X_test)

# important features
plot_importance(model)
plt.show() 

NameError: name 'XGBRegressor' is not defined

In [115]:
print('MSE:', mean_squared_error(y_test, pred_test))
print('R Square:', r2_score(y_test, pred_test))

MSE: 0.7330025861559677
R Square: -0.004759173223265023


In [113]:
#Set the different values of parameters to be tested
parameters = {'max_depth':[1,3,5,7,9],'learning_rate':[0.1,0.01],'n_estimators':[1,3,5,10,50,100],'silent':['True'],'objective':['reg:linear'],'eval_metric':['rmse']}
model = xgb.XGBRegressor()
xgb_regressor = GridSearchCV(model,parameters,scoring='neg_mean_squared_error',cv=5)
xgb_regressor.fit(X_train,y_train)
print(xgb_regressor.best_params_)
print(xgb_regressor.best_score_)


{'eval_metric': 'rmse', 'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 100, 'objective': 'reg:linear', 'silent': 'True'}
-1.0566245274841088


In [120]:
predictions = xgb_regressor.predict(X_test)

In [118]:
compare = pd.DataFrame(columns = ['pred','actual'])

In [139]:
compare['pred']=predictions
compare['actual']=y_test.reset_index(drop=True)

In [141]:
y_test

2764   -0.105463
4767   -0.105463
3814   -0.105463
3499   -0.105463
2735   -0.105463
3922   -0.105463
2701   -0.105463
1179   -0.105463
932    -0.105463
792    -0.105463
1852   -0.105463
1185   -0.105463
1724   -0.105463
4080   -0.105463
3823   -0.105463
4054   -0.105463
2721   -0.105463
3903   -0.105463
1865   -0.105463
759    -0.105463
2416   -0.105463
3670   -0.105463
4838   -0.105463
3090   -0.105463
1546   -0.105463
1132   -0.105463
4703   -0.105463
4494   -0.105463
2775   -0.105463
3185   -0.105463
          ...   
55     -0.105463
115    -0.105463
3396   -0.105463
1291   -0.105463
3451   -0.105463
2948   -0.105463
4353   -0.105463
4762   -0.105463
1720   -0.105463
2068   -0.105463
703    -0.105463
2027   -0.105463
301    -0.105463
4734   -0.105463
3936   -0.105463
560    -0.105463
1386   -0.105463
3547   -0.105463
1896   -0.105463
1535   -0.105463
4823   -0.105463
994    -0.105463
2083   -0.105463
3312   -0.105463
86     -0.105463
448    -0.105463
921    -0.105463
4087   -0.1054

In [92]:
# save the model to disk
filename = '../models/XGBRegressor.sav'
pickle.dump(xgb_regressor, open(filename, 'wb'))

In [ ]:
"""
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)
"""

In [235]:
df = pd.read_csv('../data/raw/audit_risk.csv')
df_type = pd.read_csv('../data/raw/audit_risk_column_type.csv')

In [236]:
def get_target(df,df_type):
    for c in df:
        if (column_type(c,df_type) == 'Flag_Continuous' or column_type(c,df_type) == 'Flag_Categorical'):
            return(c)

In [237]:
def column_type(column_name,df_type):
    return (df_type.loc[df_type['Variable'] == column_name, 'Type'].iloc[0])

In [238]:
get_target(df,df_type)

'Risk'

In [212]:
df_type

,Variable,Type
0,Sector_score,Continuous
1,LOCATION_ID,Nominal
2,PARA_A,Continuous
3,Score_A,Continuous
4,Risk_A,Continuous
5,PARA_B,Continuous
6,Score_B,Continuous
7,Risk_B,Continuous
8,TOTAL,Continuous
9,numbers,Continuous


In [223]:
df_type.loc[df_type['Variable'] == 'Sector_score', 'Type'].iloc[0]

'Continuous'

In [231]:

for c in df: 
    a = df_type.loc[df_type['Variable'] == c, 'Type'].iloc[0]
    print(a)

Continuous
Nominal
Continuous
Continuous
Continuous
Continuous
Continuous
Continuous
Continuous
Continuous


IndexError: single positional indexer is out-of-bounds

In [232]:
c

'Score_B.1'

In [218]:
a = []
for c in df:
    a.append(c)
len(a)

27

In [233]:
df_type.loc[df_type['Variable'] == 'Score_B.1','Type'].iloc[0]

IndexError: single positional indexer is out-of-bounds

In [174]:
df_type

,Variable,Type
0,Sector_score,Continuous
1,LOCATION_ID,Nominal
2,PARA_A,Continuous
3,Score_A,Continuous
4,Risk_A,Continuous
5,PARA_B,Continuous
6,Score_B,Continuous
7,Risk_B,Continuous
8,TOTAL,Continuous
9,numbers,Continuous


In [6]:
df1 = pd.read_csv('../data/processed/processed_kaggle_train_set.csv')
df_type1 = pd.read_csv('../data/processed/processed_kaggle_column_type.csv')

In [3]:
target_name = get_target(df1,df_type1)
X = df1.drop(columns = target_name)
y = df1[target_name]

NameError: name 'get_target' is not defined

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=1)